<a href="https://colab.research.google.com/github/messerb5467/codechef-problems/blob/main/vaccine-distrib/Codechef_covid19_vaccine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Covid 19 codechef problem
I'm working through a codechef problem [shown here](https://www.codechef.com/problems/VACCINE2) and the main benefit of writing the problem out is to focus and communicate my thoughts as I'm going through the work.

The problem is very straightforward and can be summarized as follows without going to the actual page linked above. We'll need to read and sort through text files in the following format:

```
2 -> Number of testcases present in the file.
10 1 -> Number of people in the population and the number of people the hospital can serve on a daily basis.
10 20 30 40 50 60 90 80 100 1 -> Population that the hospital needs to serve.
5 2 -> 2nd testcase.
9 80 27 72 79
```

Note that ceilings are considered in the described solutions since we cannot serve the 2 different groups of people on the same day.

### Naive unacceptable solution
At it's most basic approach, a shortcut solution is the ceiling of the 
number of people present in the population divided by the number of people we can serve per day:
```
ceiling(N/D)
```

### Revising the algorithm
We have two big groups in here, both at risk and not at risk, and we should prioritize getting the at risk through first since they are a higher risk category. Adjusting the problem we get:
```
ceiling(at_risk_pop/D) + ceiling(not_at_risk_pop/D)
```

`at_risk_pop` and `not_at_risk_pop` is simply the lengths of filtered numpy arrays which are very easy to generate. Without knowing the architecture in more detail, we also get to leverage SIMD instructions under the covers since we operate in vectorized manner which will make our solutions more scalable in the long term.

In [ ]:
from google.colab import files
files.upload()

Saving covid 19 test.txt to covid 19 test.txt


{'covid 19 test.txt': b'2\r\n10 1\r\n10 20 30 40 50 60 90 80 100 1\r\n5 2\r\n9 80 27 72 79'}

In [ ]:
import numpy as np
from math import ceil

def get_ints_from_str(string_rep, sep=" "):
  return list(map(lambda x: int(x), string_rep.split(sep)))

def treat_patients(testcase_file):
  with open(testcase_file) as f:
    num_testcases = int(f.readline())
    # Keep a memory efficient solution so I'm preferring to keep the file 
    # descriptor open longer. Nothing else is trying to use it anyways, so
    # no big deal on that front.
    for i in range(0, num_testcases):
      _, D = get_ints_from_str(f.readline()) ## Always len 2
      # We get into a sticking point here based on how we should filter
      # our data https://stackoverflow.com/questions/58422690/filtering-a-numpy-array
      # Reviewing the provided graphs in the answer, the speed is still 
      # sufficient (1ms turn around time at the worst case).
      #
      # Similarly, if we notice this methodology taking some time, we can
      # profile it and get a sense of how much memory we're using based on this:
      # https://stackoverflow.com/questions/11784329/python-memory-usage-of-numpy-arrays
      population = np.array(get_ints_from_str(f.readline()))
      at_risk = (population <= 9) | (population >= 80)
      at_risk_pop = population[at_risk]
      not_at_risk_pop = population[~at_risk]
      total_days = ceil(len(at_risk_pop) / D) + ceil(len(not_at_risk_pop) / D)
      print(total_days)


In [ ]:
treat_patients('covid 19 test.txt')

10
3
